In [1]:
from IPython.display import display
import joblib
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score

In [2]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.preprocessing._encoders')

## 1. Obtenção de dados

In [3]:
df = pd.read_csv('../data/raw/wines_SPA.csv')
df_dict = pd.read_csv('../data/external/dict.csv')

In [4]:
df_dict

,variavel,descricao,tipo,subtipo
0,winery,Vinícola: local onde o vinho foi produzido,qualitativa,nominal
1,wine,vinho,qualitativa,nominal
2,year,ano,quantitativa,discreta
3,rating,avaliação: de 4.2 a 4.9,quantitativa,discreta
4,num_reviews,número de avaliações,quantitativa,discreta
5,country,país de origem,qualitativa,nominal
6,region,região do vinho,qualitativa,nominal
7,price,preço em euros,quantitativa,continua
8,type,tipo do vinho,qualitativa,nominal
9,body,"valor do ""corpo"" do vinho: de 2 a 5",quantitativa,discreta


In [5]:
# a coluna year está como string, mudo para numérico
df['year'] = df['year'].replace('N.V.', -1)
df['year'] = df['year'].replace(np.nan, -1)
df['year'] = df['year'].astype(int)
df['year'] = df['year'].replace(-1, np.nan)

## 2. Preparação de dados

In [6]:
target_column = 'wine'
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)
continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)
discrete_columns = (
    df_dict
    .query("subtipo == 'discreta'")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

In [7]:
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')), # codificação de variáveis
    ('normalization', MinMaxScaler()) # normalização de dados
])
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    ('normalization', MinMaxScaler()) # normalização de dados
])
discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='median')), # tratamento de dados faltantes
    ('normalization', MinMaxScaler()) # normalização de dados
])
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('discrete', discrete_preprocessor, discrete_columns)
])

model = LogisticRegression()

## 3. Seleção de modelos

In [8]:
# experiment settings
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = .2
random_state = 42
max_iter = 1000
scoring = 'accuracy'
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='macro', zero_division=0),
    'recall': make_scorer(recall_score, average='macro', zero_division=0),
    'f1': make_scorer(f1_score, average='macro', zero_division=0)
}
# model settings
models = [
    ('K-Nearest Neighbors', KNeighborsClassifier(), {"n_neighbors": range(3, 20, 2), 'weights': ['distance', 'uniform']}),
    # ('Suport Vector Machines', SVC(random_state=random_state), {"kernel": ["linear", "rbf"], 'C':[0.1,1,10,100, 1000],'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}),
    ('Decision Tree',  DecisionTreeClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8]}),
    ('Random Forest',  RandomForestClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8], 'n_estimators': [10, 30]}),
]

In [ ]:
results = pd.DataFrame({})
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)
for model_name, model_object, model_parameters in models:
    print(f"running {model_name}...")
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    scores = cross_validate(
        estimator=approach,
        X=X,
        y=y,
        cv=cross_validate_comparative_analysis,
        scoring=metrics
    )
    
    scores['model_name'] = [model_name] * n_splits_comparative_analysis
    scores_df = pd.DataFrame(scores)
    model_names = scores_df['model_name']
    scores_df = scores_df.drop(columns=['model_name'])
    scores_df = scores_df.agg(['mean', 'std'])
    scores_df['model_name'] = model_name
    display(scores_df)
    
    results = pd.concat([results,pd.DataFrame(scores)], ignore_index=True)

running K-Nearest Neighbors...


,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1,model_name
mean,6.144364,0.195152,0.871400,0.347486,0.373027,0.351125,K-Nearest Neighbors
std,1.762592,0.089114,0.008107,0.026856,0.022936,0.025305,K-Nearest Neighbors


running Decision Tree...


,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1,model_name
mean,2.145434,0.040535,0.768000,0.104927,0.123312,0.107596,Decision Tree
std,0.187752,0.003956,0.010927,0.004174,0.003734,0.003732,Decision Tree


running Random Forest...


### 3.1 Resultados gerais

In [ ]:
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.nanmin(s.values), props, '')
        return np.where(s == np.nanmax(s.values), props, '')

display(
    results
    .groupby('model_name')
    .agg(['mean', 'std']).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

### 3.2 Persistência do Modelo

In [ ]:
#Obtem o modelo e os parametros ganhadores
model_name, model_object, model_parameters  = [foo for foo in models if foo[0] == "K-Nearest Neighbors"][0] 


model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )

approach = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model_grid_search)
])

approach.fit(X, y) #Seleciona o approach

print(f"Hiper parâmetros do modelo: {approach.steps[1][1].best_params_}")

In [ ]:
joblib.dump(approach, '../models/model.joblib') # Salva o modelo em disco